In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:

# Load the data
data = pd.read_csv('../data/trainData/solar power and weather condition.csv')


features = ['Hour', 'Solar Zenith Angle', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'Relative Humidity', 'DHI', 'DNI', 'GHI']
target = 'Energy Output(kWh)'

In [8]:


data_features = data[features + [target]]


X = data_features[features].values
y = data_features[target].values


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Add dimension for output


X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)


In [12]:
X_train.shape

torch.Size([11520, 9])

In [6]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Initialize hidden states with correct batch size
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), hidden_size).to(x.device)
        
        # Forward pass
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


# Parameters
input_size = 1  # Number of features
hidden_size = 50
num_layers = 2

# Instantiate model
model = LSTMModel(input_size, hidden_size, num_layers)
